# Q.5

### (Unsupervised labeling) This exercise concerns the classdemo.py file shared with you. We saw that EM algorithm learnt the hidden parameters fairly well. This question asks you to classify every point to each coin. Record the error rate of this “classifier”.


### Importing Libraries

In [4]:
import numpy as np
# library, which provides a progress bar that shows the progress of the loop
from tqdm import tqdm

### Parameters
1. theta_A and theta_B are the bias values of two coins in a Bernoulli distribution
2. N is the number of times that a coin is flipped and the outcome (either "head" or "tail") is recorded i.e number of trials.
3. D is the total number of trials or flips of a coin in a Bernoulli distribution
4. x[i] represents the number of "head" outcomes observed in the i-th trial or flip of a coin.
5. z is to store the hidden label that indicates which coin was used to generate the data for each observation. z[i] is 0 if coin A was used for the i-th observation, and 1 if coin B was used.


### Generating Data

In [5]:
def data_generation(N, D, bias_A, bias_B):
    # Initialize arrays to store the generated data
    x = np.zeros(N)  # Number of heads
    z = np.zeros(N)  # Hidden label (0 for A, 1 for B)

    # Loop over the samples and generate data
    for i in range(N):
        # Randomly choose between coin A and coin B
        if np.random.uniform() < 0.5:
            # Generate data from coin A and store the number of heads
            x[i] = np.random.binomial(D, bias_A)
        else:
            # Generate data from coin B and store the number of heads
            x[i] = np.random.binomial(D, bias_B)
            # Set the hidden label to 1 to indicate that coin B was used
            z[i] = 1

    return x, z

### Parameter Values as per Classdemo.ipynb

In [6]:
bias_A = 0.4  # Bias of coin A
bias_B = 0.6  # Bias of coin B
D = 100  # X's, D represents the number of tosses in each experiment
N = 10000  # Total Samples


### Generate Number of Heads and Hidden Label (0 for A, 1 for B)

In [7]:
x, z = data_generation(N, D, bias_A, bias_B)

## Expectation maximisation of Bernoulli trails

The probability of a Bernoulli distribution in given by  
$$p(\textbf{x}|\boldsymbol{\mu}) = \prod_{i=1}^{784}\mu_i^{x_i}(1-\mu_i)^{(1-x_i)}$$
$$p(x|\mu) = \mu^{x}(1-\mu)^{(D-x)}$$


## E-step

Computing the responsibilities using the current parameter values. Probability of coin A or B for each trial, $\pi_A=\pi_B=0.5$

$$ \gamma(z_{nA}) = \frac{\pi_Ap(x_n|\mu_A)}{ \pi_Ap(x_n|\mu_A) + \pi_Bp(x_n|\mu_B) } $$

$$ \gamma(z_{nA}) = \frac{\pi_A\mu_A^{x_n}(1-\mu_A)^{(D-x_n)}}{ \pi_A\mu_A^{x_n}(1-\mu_A)^{(D-x_n)} + \pi_B\mu_B^{x_n}(1-\mu_B)^{(D-x_n)} } $$

$$ \gamma(z_{nB}) = 1 - \gamma(z_{nA})$$

## M - step

Re-estimate the parameters using the current responsibilities  
$$\mu_A^{new}=\frac{1}{N_A}\sum_{n=1}^{N}\gamma(z_{nA})x_n,\space\space\space\space\space\space\space \mu_B^{new}=\frac{1}{N_B}\sum_{n=1}^{N}\gamma(z_{nB})x_n, $$  

$$\pi_A^{new}=\frac{N_A}{N} \space\space\space\space\space\space\space \pi_B^{new}=\frac{N_B}{N}$$  

where,  
$$N_A=\sum_{n=1}^N\gamma(z_{nA}) \space\space\space\space\space\space\space N_B=\sum_{n=1}^N\gamma(z_{nB})$$

### Estimating the parameters of two coins A and B that were used to generate a dataset x of length N, where each x_i is the number of heads observed in D tosses of the i-th coin.

In [8]:
# Function Returns:

# theta_A: float representing the estimated bias of coin A
# theta_B: float representing the estimated bias of coin B

def Expectation_Maximization_Algorithm(x, D, N, repeatcount):

    # INITIALIZE BIAS FOR COIN A AND COIN B
    theta_A = 0.51  # beta or uniform choices are ok
    theta_B = 0.534
    gamma_A = np.zeros(N)
    gamma_B = np.zeros(N)
    for i in tqdm(range(repeatcount)):
        # print(i)

        #********************************#
        # EStep starts
        #********************************#
        
        # In the E step, we estimate the "responsibilities" of each coin
        # for generating each experiment. We calculate gamma_A[j] and gamma_B[j]
        # for each experiment j as per the formulas given in Bishop's book
        
        for j in range(N):
            gamma_A[j] = 0.5 * np.power(theta_A, x[j]) * np.power(1-theta_A, D-x[j])
            gamma_A[j] /= (0.5*np.power(theta_A, x[j]) * np.power(1-theta_A, D-x[j]) + 0.5 * np.power(theta_B, x[j]) * np.power(1-theta_B, D-x[j]))
            gamma_B[j] = 1 - gamma_A[j]

        # ********************************#
        # EStep ends
        # ********************************#

        # In the M step, we update the estimates of the biases of the coins
        # using the estimated responsibilities (gamma_A and gamma_B) calculated
        # in the E step. We calculate the new values of theta_A and theta_B
        # using the formulas given in Bishop's book

        # ********************************#
        # MStep starts
        # ********************************#
        theta_A_numerator = theta_A_denominator = theta_B_numerator = theta_B_denominator = 0
        for k in range(N):
            theta_A_numerator += gamma_A[k]*x[k]
            theta_A_denominator += gamma_A[k]*D
            theta_B_numerator += gamma_B[k]*x[k]
            theta_B_denominator += gamma_B[k]*D
        theta_A = theta_A_numerator/theta_A_denominator
        theta_B = theta_B_numerator/theta_B_denominator
        # ********************************#
        # MStep ends
        # ********************************#

    return theta_A, theta_B

### Computing theta's i.e bias values of two coins in a Bernoulli distribution using EM Algorithm

In [9]:
num_iterations = 1000
theta_A, theta_B = Expectation_Maximization_Algorithm(x, D, N, num_iterations)
print("Predicted Biases of Coin A and B are respectively: ", theta_A, theta_B)

100%|██████████| 1000/1000 [01:21<00:00, 12.32it/s]

Predicted Biases of Coin A and B are respectively:  0.4004784920169179 0.6005202165558978


### Bayes Classifier Implementation

In [10]:
def bayes_classifier(x, theta_A, theta_B, N):

    # classified_list contains the classification results where 1 represents classification to coin B and 0 represents classification to coin A

    classified_list = np.zeros(N)
    for i in range(N):
        # Calculate the probabilities of the observation belonging to each of the coins
        
        # probability of getting xi given A
        prob_A_D = np.multiply(np.power(theta_A, x[i]), np.power(1-theta_A,D-x[i]))
        # probability of getting xi given B
        prob_B_D = np.multiply(np.power(theta_B, x[i]), np.power(1-theta_B,D-x[i]))

        # Classify the observation as belonging to either coin A or coin B based on the
        # probability calculation

        if(prob_A_D < prob_B_D):
            classified_list[i] = 1

    # return the classification results
    return classified_list

### Predictions

In [11]:
predicted_class = bayes_classifier(x, theta_A, theta_B, N)

### Accuracy

In [12]:
# true_data : original classification of the observed data, where 1 represents classification to coin B and 0 represents classification to coin A
# predicted_data: the predicted classification of the observed data, where 1 represents classification to coin B and 0 represents classification to coin A

def compute_accuracy(true_data, predicted_data, N):
    incorrect_counter = 0
    # Loop through the original and predicted classifications and count the number of incorrect
    # classifications
    for i in range(N):
        if(true_data[i]!=predicted_data[i]):
            incorrect_counter+=1
    # Calculate the accuracy of the classification as a percentage
    return (1-(incorrect_counter)/N)*100

In [13]:
accuracy = compute_accuracy(z, predicted_class, N)
print("The accuracy is: ", accuracy)

The accuracy is:  97.78999999999999
